In [ ]:
!pip install coremltools

In [1]:
import coremltools as ct
from PIL import Image

Torch version 2.5.1 has not been tested with coremltools. You may run into unexpected errors. Torch 1.13.1 is the most recent version that has been tested.


In [4]:
model = ct.models.MLModel('/Users/selimgilon/Desktop/SportBoost/yolo-flutter/yolo-flutter-app/example/assets/yolov8n-pose.mlmodel')

FileNotFoundError: [Errno 2] No such file or directory: "/Users/selimgilon/Desktop/SportBoost/YOLO11/y.png'"

In [7]:
model

input {
  name: "image"
  type {
    imageType {
      width: 640
      height: 640
      colorSpace: RGB
    }
  }
}
output {
  name: "var_1035"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  shortDescription: "Ultralytics YOLOv8n-pose model trained on /usr/src/app/ultralytics/datasets/coco-pose.yaml"
  versionString: "8.3.53"
  author: "Ultralytics"
  license: "AGPL-3.0 License (https://ultralytics.com/license)"
  userDefined {
    key: "batch"
    value: "1"
  }
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==2.5.1+cu121"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "6.2"
  }
  userDefined {
    key: "date"
    value: "2024-12-22T18:32:51.438041"
  }
  userDefined {
    key: "docs"
    value: "https://docs.ultralytics.com"
  }
  userDefined {
    key: "imgsz"
    value: "[640, 640]"
  }
  userDefined {
    key: "kpt_shape"
    value: "[17, 3]"
  }
  userDefined {
    key: "names"
    

In [22]:
import numpy as np
from PIL import Image, ImageDraw

# Load and preprocess the image
img = Image.open("s.png").convert("RGB").resize((640, 640))

# Make a prediction
spec = model.get_spec()
model_input_name = spec.description.input[0].name
model_output_name = spec.description.output[0].name

coreml_out = model.predict({model_input_name: img})
coreml_output_tensor = coreml_out[model_output_name]

# Print the shape of the output tensor
print("Shape of CoreML output: ", coreml_output_tensor.shape)

# Parameters
confidence_threshold = 0.2
iou_threshold = 0.5

# Helper function to compute IoU
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

# Parse the output and filter by confidence
detections = coreml_output_tensor[0]  # Shape: (56, 8400)
num_detections = detections.shape[1]

filtered_detections = []

for detection_index in range(num_detections):
    detection = detections[:, detection_index]

    # Extract bounding box coordinates (center_x, center_y, width, height)
    center_x, center_y, width, height = detection[:4]

    # Extract confidence score
    confidence = detection[4]

    if confidence < confidence_threshold:
        continue  # Skip low-confidence detections

    # Bounding box coordinates
    bbox_x1 = (center_x - width / 2)
    bbox_y1 = (center_y - height / 2)
    bbox_x2 = (center_x + width / 2)
    bbox_y2 = (center_y + height / 2)

    # Extract keypoints (17 keypoints × (x, y, confidence))
    keypoints = detection[5:].reshape(-1, 3)
    filtered_detections.append({
        "bbox": [bbox_x1, bbox_y1, bbox_x2, bbox_y2],
        "confidence": confidence,
        "keypoints": keypoints
    })

# Apply Non-Maximum Suppression (NMS)
filtered_detections.sort(key=lambda x: x["confidence"], reverse=True)  # Sort by confidence
final_detections = []

while filtered_detections:
    best_detection = filtered_detections.pop(0)
    final_detections.append(best_detection)

    filtered_detections = [
        det for det in filtered_detections
        if compute_iou(best_detection["bbox"], det["bbox"]) < iou_threshold
    ]

# Draw keypoints and bounding boxes on the image
draw = ImageDraw.Draw(img)

for det in final_detections:
    bbox = det["bbox"]
    keypoints = det["keypoints"]

    # Draw bounding box
    draw.rectangle(bbox, outline="red", width=2)

    # Draw keypoints
    for x, y, conf in keypoints:
        if conf > confidence_threshold:
            draw.ellipse([(x - 2, y - 2), (x + 2, y + 2)], fill="black")

# Show the image with keypoints and bounding boxes
img.show()


Shape of CoreML output:  (1, 56, 8400)


In [14]:
coreml_output_tensor

array([[[9.49218750e+00, 1.74375000e+01, 2.41718750e+01, ...,
         5.26500000e+02, 5.43500000e+02, 5.74500000e+02],
        [1.95156250e+01, 1.71250000e+01, 1.17890625e+01, ...,
         5.65000000e+02, 4.98750000e+02, 4.57250000e+02],
        [2.74687500e+01, 4.04375000e+01, 5.16875000e+01, ...,
         2.36250000e+02, 2.36250000e+02, 1.47000000e+02],
        ...,
        [5.89355469e-01, 1.37890625e+01, 1.80156250e+01, ...,
         5.47000000e+02, 5.77000000e+02, 5.95000000e+02],
        [2.78281250e+01, 2.66093750e+01, 2.50781250e+01, ...,
         5.88000000e+02, 5.93000000e+02, 6.02500000e+02],
        [9.57031250e-02, 1.24755859e-01, 1.25732422e-01, ...,
         9.76562500e-02, 6.90917969e-02, 5.54199219e-02]]], dtype=float32)